## ETL Development

This file serves as as draft for designing & development of project ETLs - transformations & preprocessing of data for modeling. The ETL should be part of runnable  
pipeline for model encapsulation & enviroment isolation purposes (leak of statistics).

Includes:
- Transformation ETLs
- Encoding ETLs

ETLs must be part of pipeline to ensure statistical calculation only on train dataset.

Objetive of the ETL pipeline is to transform input dataset into model ready form.

In [10]:
import os
import sys

sys.dont_write_bytecode =True

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings('ignore')

loading data

In [11]:
DATA_DIR = './Data/'
FILE = 'spotify_tracks_kaggle_weekly.csv'

In [12]:
data = pd.read_csv(DATA_DIR + FILE)

In [13]:
data.min()

track_id                                       0003ilHWJH7c5UtjKLPiC9
track_name                  "All That Is or Ever Was or Ever Will Be"
artist_name                       "Karaoke - Britney Spears, Madonna"
year                                                             1971
popularity                                                          0
artwork_url                                      No artwork available
album_name                                    "...von Bach bis AC/DC"
acousticness                                                     -1.0
danceability                                                     -1.0
duration_ms                                                    5000.0
energy                                                           -1.0
instrumentalness                                                 -1.0
key                                                              -1.0
liveness                                                         -1.0
loudness            

columns to be dropped:

- track_id : ID
- artwork_url : url data - not intended for modelling purposes
- track_url : url data - not intended for modelling purposes
- track_name : user-friendly ID - not intended for modelling purposes (could be used in NLP analysis, however that will not be subject of this project)

In [14]:
DROP_COLUMNS = ['track_id', 'artwork_url', 'track_url', 'track_name']

In [15]:
data = data.drop(DROP_COLUMNS, axis=1, errors='ignore')

In [16]:
data.head()

,artist_name,year,popularity,album_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,language
0,Anirudh Ravichander,2024,59,"Leo Das Entry (From ""Leo"")",0.0241,0.753,97297.0,0.970,0.055300,8.0,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,Tamil
1,"Anirudh Ravichander, Pravin Mani, Vaishali Sri...",2024,47,AAO KILLELLE,0.0851,0.780,207369.0,0.793,0.000000,10.0,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,Tamil
2,"Anirudh Ravichander, Anivee, Alvin Bruno",2024,35,Mayakiriye Sirikiriye (Orchestral EDM),0.0311,0.457,82551.0,0.491,0.000000,2.0,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,Tamil
3,"Anirudh Ravichander, Bharath Sankar, Kabilan, ...",2024,24,Scene Ah Scene Ah (Experimental EDM Mix),0.2270,0.718,115831.0,0.630,0.000727,7.0,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,Tamil
4,"Anirudh Ravichander, Benny Dayal, Leon James, ...",2024,22,Gundellonaa X I Am a Disco Dancer (Mashup),0.0153,0.689,129621.0,0.748,0.000001,7.0,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,Tamil


### Encoding

Due to the nature of the dataset multiple categorical (numerical as well) columns will require specific encoding method to convey information correctly.

- Categorical data:
    - artist_name
    - album_name
    - language


- Numerical data:
    - key
    - mode

- Columns which can be encoded by OneHot principle:
    - language
    - artist_name (maybe but avg. artist popularity could be as well)

Defining ETL model structure

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

In [18]:
class ETL(BaseEstimator, TransformerMixin):
    '''Custom ETL model, abstract class, when creating an ETL use class inheritance and overwrite
       self.transform method with custom transformation, must return transformed X.'''

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X

Artist & Albums COLUMN encoding

- problem: multiple artists in one string -> separated by ','
- solutions:
    - nested average: sum(averages of artists) / len(artists)
    - frequency

8491 unique aritst names ... OneHot/Dummy encoding is not feasible

In [19]:
data['artist_name'].str.split(',').explode().nunique()

8491

Dedicated COLUMN ETL

In [20]:
class ArtistEncoding(ETL):

    SEPARATOR = ','
    TARGET_COLUMN = 'artist_name'
    Y_COLUMN = 'popularity'

    def transform(self, X, y=None):
        X_new = X.copy()

        # popularity aggregation method
        X_new = self.aggregate_popularity(X_new)

        return X_new
    

    def aggregate_popularity(self, X):
        
        X['artist_name_split'] = X[self.TARGET_COLUMN].str.split(self.SEPARATOR)
        
        artist_popularity = X.explode('artist_name_split').groupby('artist_name_split')[self.Y_COLUMN].mean()

        def encode(artists, values):
            popularities = [values.get(artist, 0) for artist in artists]
            return sum(popularities) / len(popularities) if popularities else 0


        X['artist_name_encoded'] = X['artist_name_split'].apply(lambda artists: encode(artists, artist_popularity))

        return X

In [21]:
ArtistEncoding().fit_transform(data)

,artist_name,year,popularity,album_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,language,artist_name_split,artist_name_encoded
0,Anirudh Ravichander,2024,59,"Leo Das Entry (From ""Leo"")",0.02410,0.753,97297.0,0.970,0.055300,8.0,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,Tamil,[Anirudh Ravichander],22.035066
1,"Anirudh Ravichander, Pravin Mani, Vaishali Sri...",2024,47,AAO KILLELLE,0.08510,0.780,207369.0,0.793,0.000000,10.0,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,Tamil,"[Anirudh Ravichander, Pravin Mani, Vaishali ...",26.289466
2,"Anirudh Ravichander, Anivee, Alvin Bruno",2024,35,Mayakiriye Sirikiriye (Orchestral EDM),0.03110,0.457,82551.0,0.491,0.000000,2.0,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,Tamil,"[Anirudh Ravichander, Anivee, Alvin Bruno]",19.149466
3,"Anirudh Ravichander, Bharath Sankar, Kabilan, ...",2024,24,Scene Ah Scene Ah (Experimental EDM Mix),0.22700,0.718,115831.0,0.630,0.000727,7.0,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,Tamil,"[Anirudh Ravichander, Bharath Sankar, Kabila...",15.742419
4,"Anirudh Ravichander, Benny Dayal, Leon James, ...",2024,22,Gundellonaa X I Am a Disco Dancer (Mashup),0.01530,0.689,129621.0,0.748,0.000001,7.0,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,Tamil,"[Anirudh Ravichander, Benny Dayal, Leon Jame...",17.775819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62312,"Arvind Raj, Sheezay, Music Kitchen, FSPROD Vin...",2021,30,Sani (G.O.A.T Remix),0.08490,0.877,359718.0,0.469,0.000000,7.0,0.1100,-11.458,1.0,0.3530,100.024,4.0,0.504,Tamil,"[Arvind Raj, Sheezay, Music Kitchen, FSPROD...",29.662500
62313,"A H Kaashif, Navakkarai Naveen Prabanjam, Asal...",2021,30,Bachelor (Original Motion Picture Soundtrack),0.07880,0.833,154364.0,0.689,0.000000,8.0,0.2670,-5.061,1.0,0.0813,110.041,4.0,0.524,Tamil,"[A H Kaashif, Navakkarai Naveen Prabanjam, A...",29.375000
62314,"Rakesh Ambigapathy, Asal Kolaar, MC Vickey",2021,18,Yo Baby,0.00918,0.842,234000.0,0.765,0.000000,7.0,0.0790,-5.855,1.0,0.1370,122.038,4.0,0.669,Tamil,"[Rakesh Ambigapathy, Asal Kolaar, MC Vickey]",16.788978
62315,"Asal Kolaar, Priyadarshan Balasubramanian",2021,1,Arjuna Phalguna,0.10100,0.682,180807.0,0.861,0.000000,1.0,0.3200,-6.061,1.0,0.2850,74.993,4.0,0.439,Tamil,"[Asal Kolaar, Priyadarshan Balasubramanian]",11.653846


Frequency encoding

In [117]:
class FrequencyEncoder(ETL):

    TARGET_COL = 'artist_name'

    def __init__(self, strategy='max'):
        self.frequencies = None
        self.default_value = 0
        self.strategy = strategy
    
    def fit(self, X, y=None):
        freqs = X[self.TARGET_COL].str.split(',').explode().value_counts()
        self.frequencies = freqs.to_dict()
        return self
    

    def transform(self, X, y=None):

        X_new = X.copy()

        if self.strategy == 'max':
            X_new[self.TARGET_COL] = X[self.TARGET_COL].str.split(',').apply(
                lambda artists: max(self.frequencies.get(artist, self.default_value) for artist in artists)
            )

        elif self.strategy == 'sum':
            X_new[self.TARGET_COL] = X[self.TARGET_COL].str.split(',').apply(
                lambda artists: sum(self.frequencies.get(artist, self.default_value) for artist in artists)
            )

        #X_new[self.TARGET_COL] = X[self.TARGET_COL].map(self.frequencies).fillna(self.default_value)

        return X_new


Key column transformation - cyclical data capturing:

Original mapping is linear - relationship of the keys is not correctly represented, according to the data model description  
the mapping represents the keys follows:

$$
\begin{array}{|c|c|}
\hline
\textbf{Key} & \textbf{Pitch Class} \\ \hline
0 & C \\ \hline
1 & C\# / D\flat \\ \hline
2 & D \\ \hline
3 & D\# / E\flat \\ \hline
4 & E \\ \hline
5 & F \\ \hline
6 & F\# / G\flat \\ \hline
7 & G \\ \hline
8 & G\# / A\flat \\ \hline
9 & A \\ \hline
10 & A\# / B\flat \\ \hline
11 & B \\ \hline
\end{array}
$$

We will first transform the original column to the correct representation on the circle of fifths, then remap the correct ordinal representation  
to the circle of fifths trigonometric representation defined as:

$\theta = \frac{2\pi}{12}.key$

Then we can create correct encoding representation with two columns defined as:

$x = \cos(\theta)$, $y = \sin(\theta)$

In [23]:
class CircleOfFifthsEncoding(ETL):

    TARGET_COLUMN = 'key'

    # Pôvodný mapping z datasetu (chromatic scale: 0–11)
    # Tento mapping je založený na poltónových intervaloch.
    chromatic_scale = {
        0: "C",
        1: "C#/Db",
        2: "D",
        3: "D#/Eb",
        4: "E",
        5: "F",
        6: "F#/Gb",
        7: "G",
        8: "G#/Ab",
        9: "A",
        10: "A#/Bb",
        11: "B"
    }

    # Korektný ordinal pre Circle of Fifths:
    # Táto sekvencia reflektuje harmonické vzťahy medzi kľúčmi:
    # každý nasledujúci kľúč je vzdialený o kvintu (7 poltónov).
    circle_of_fifths_mapping = {
        0: 0,  # C
        7: 1,  # G
        2: 2,  # D
        9: 3,  # A
        4: 4,  # E
        11: 5, # B
        6: 6,  # F#/Gb
        1: 7,  # C#/Db
        8: 8,  # G#/Ab
        3: 9,  # D#/Eb
        10: 10, # A#/Bb
        5: 11  # F
    }


    def transform(self, X, y=None):

        X_new = X.copy()

        ordinal_remapping = X[self.TARGET_COLUMN].map(self.circle_of_fifths_mapping)

        theta = (2 * np.pi / 12) * ordinal_remapping

        x = np.cos(theta)
        y = np.sin(theta)

        X_new = X_new.drop(self.TARGET_COLUMN, axis=1, errors='ignore')

        X_new['key_x'] = x
        X_new['key_y'] = y

        return X_new

In [24]:
CircleOfFifthsEncoding().fit_transform(data).head()

,artist_name,year,popularity,album_name,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,time_signature,valence,language,key_x,key_y
0,Anirudh Ravichander,2024,59,"Leo Das Entry (From ""Leo"")",0.0241,0.753,97297.0,0.970,0.055300,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,Tamil,-0.500000,-0.866025
1,"Anirudh Ravichander, Pravin Mani, Vaishali Sri...",2024,47,AAO KILLELLE,0.0851,0.780,207369.0,0.793,0.000000,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,Tamil,0.500000,-0.866025
2,"Anirudh Ravichander, Anivee, Alvin Bruno",2024,35,Mayakiriye Sirikiriye (Orchestral EDM),0.0311,0.457,82551.0,0.491,0.000000,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,Tamil,0.500000,0.866025
3,"Anirudh Ravichander, Bharath Sankar, Kabilan, ...",2024,24,Scene Ah Scene Ah (Experimental EDM Mix),0.2270,0.718,115831.0,0.630,0.000727,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,Tamil,0.866025,0.500000
4,"Anirudh Ravichander, Benny Dayal, Leon James, ...",2024,22,Gundellonaa X I Am a Disco Dancer (Mashup),0.0153,0.689,129621.0,0.748,0.000001,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,Tamil,0.866025,0.500000


### Standardization - Scaling

In [25]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

We will use native sklearn scalers such as:
- standard scaler
- min max scaler

### Filtration of columns

- handling of utility columns during transformation sub-steps & EDA drop columns

In [70]:
class FilterColumns(ETL):
    
    def __init__(self, drop_columns=None):

        if drop_columns is None:
            drop_columns = []

        self.drop_columns = drop_columns


    def transform(self, X, y=None):
        X_new = X.copy()
        X_new = X.drop(self.drop_columns, axis=1, errors='ignore')
        return X_new

### NaN Handling

We will utilize KNN Imputer/Dropping Nulls

- NaNs are represented by -1

In [27]:
data.min()

artist_name         "Karaoke - Britney Spears, Madonna"
year                                               1971
popularity                                            0
album_name                      "...von Bach bis AC/DC"
acousticness                                       -1.0
danceability                                       -1.0
duration_ms                                      5000.0
energy                                             -1.0
instrumentalness                                   -1.0
key                                                -1.0
liveness                                           -1.0
loudness                                      -100000.0
mode                                               -1.0
speechiness                                        -1.0
tempo                                              -1.0
time_signature                                     -1.0
valence                                            -1.0
language                                        

In [28]:
nan_columns = ['acousticness', 'danceability', 'energy', 'liveness', 'speechiness', 'tempo', 'valence']

In [29]:
null_replaced = data[nan_columns].replace(-1, np.nan)

In [30]:
class ConvertNull(ETL):
    
    def __init__(self, columns=None, input_value=-1, output_value=np.nan):

        if columns is None:
            self.columns = []
        else:
            self.columns = columns

        self.input_value = input_value
        self.output_value = output_value

    
    def transform(self, X):
        X_new = X.copy()
        X_new[self.columns] = X_new[self.columns].replace(self.input_value, self.output_value)
        return X_new

### ETL Pipeline Construction

NOTE: akurat pozeram ze Artist popularity aggregation nieje mozne vyuzit... popularita je target parameter ktori budeme predikovat cize pri test sete by nebolo ako vypocitat  
tie priemery... nechapem ako mi to nedoslo skorej.

In [31]:
import scipy.stats

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.preprocessing import PolynomialFeatures

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV

pipeline construction

In [32]:
data.columns

Index(['artist_name', 'year', 'popularity', 'album_name', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'language'],
      dtype='object')

In [33]:
numeric_columns = list(data.columns[data.dtypes != object].difference(['key', 'mode', 'popularity']))
categorical_columns = list(data.columns[data.dtypes == object])
acustic_categorical_columns = ['key', 'mode']

nan_columns = ['acousticness', 'danceability', 'energy', 'liveness', 'speechiness', 'tempo', 'valence']
drop_columns = ['artist_name_split', 'track_id', 'artwork_url', 'track_url', 'track_name']

target_column = 'popularity'

column subset preparation

In [34]:
TARGET = 'popularity'

ONEHOT_COLUMNS = ['language']
CIRCLE_OF_FIFHTS_COLUMNS = ['key']
ARTIST_NAME = ['artist_name']

ALL_CATEGORICAL = ONEHOT_COLUMNS + CIRCLE_OF_FIFHTS_COLUMNS

NUMERIC_COLUMNS = numeric_columns
NAN_COLUMNS = nan_columns
DROP_COLUMNS = drop_columns

In [124]:
X = data.drop(target_column, axis=1)
y = data[target_column]

In [125]:
convert_null = ColumnTransformer(transformers=[
    ('conversion', ConvertNull(), NAN_COLUMNS)
], remainder='passthrough')

In [126]:
numeric_pipeline = Pipeline(steps=[
    ('scaling', StandardScaler())
])

In [127]:
artist_name_pipeline = Pipeline(steps=[
    ('encoding', FrequencyEncoder()),
    ('scaling', StandardScaler())
])

In [128]:
transformations = ColumnTransformer(transformers=[
    
    ('onehot_encoding', OneHotEncoder(sparse_output=False), ONEHOT_COLUMNS),
    ('trigonometric_encoding', CircleOfFifthsEncoding(), CIRCLE_OF_FIFHTS_COLUMNS),
    ('artist_encoding', artist_name_pipeline, ARTIST_NAME),
    ('nummeric_processing', numeric_pipeline, NUMERIC_COLUMNS)

], remainder='drop')

In [132]:
preprocessing = Pipeline(steps=[
    ('null_values', ConvertNull(columns=NAN_COLUMNS)),
    ('transformation', transformations)
])

In [120]:
X['artist_name'].str.split(',')

0                                    [Anirudh Ravichander]
1        [Anirudh Ravichander,  Pravin Mani,  Vaishali ...
2             [Anirudh Ravichander,  Anivee,  Alvin Bruno]
3        [Anirudh Ravichander,  Bharath Sankar,  Kabila...
4        [Anirudh Ravichander,  Benny Dayal,  Leon Jame...
                               ...                        
62312    [Arvind Raj,  Sheezay,  Music Kitchen,  FSPROD...
62313    [A H Kaashif,  Navakkarai Naveen Prabanjam,  A...
62314       [Rakesh Ambigapathy,  Asal Kolaar,  MC Vickey]
62315         [Asal Kolaar,  Priyadarshan Balasubramanian]
62316    [Harshavardhan Rameshwar,  G. V. Prakash,  Dha...
Name: artist_name, Length: 62317, dtype: object

In [138]:
preprocessing

Pipeline(steps=[('null_values',
                 ConvertNull(columns=['acousticness', 'danceability', 'energy',
                                      'liveness', 'speechiness', 'tempo',
                                      'valence'])),
                ('transformation',
                 ColumnTransformer(transformers=[('onehot_encoding',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['language']),
                                                 ('trigonometric_encoding',
                                                  CircleOfFifthsEncoding(),
                                                  ['key']),
                                                 ('artist_encoding',
                                                  Pipeline(steps=[('encoding',
                                                                   FrequencyEncoder()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['artist_name']),
                                                 ('nummeric_processing',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler())]),
                                                  ['acousticness',
                                                   'danceability',
                                                   'duration_ms', 'energy',
                                                   'instrumentalness',
                                                   'liveness', 'loudness',
                                                   'speechiness', 'tempo',
                                                   'time_signature', 'valence',
                                                   'year'])]))])

In [136]:
preprocessing.fit_transform(X)

array([[ 0.        ,  0.        ,  0.        , ...,  0.2843186 ,
        -0.14121875,  0.99264175],
       [ 0.        ,  0.        ,  0.        , ..., -1.70511351,
         1.2379582 ,  0.99264175],
       [ 0.        ,  0.        ,  0.        , ...,  0.2843186 ,
         0.38835472,  0.99264175],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.2843186 ,
         0.65885627,  0.6816009 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.2843186 ,
        -0.21741637,  0.6816009 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.2843186 ,
         1.2989163 ,  0.6816009 ]])

In [131]:
pd.DataFrame(transformations.fit_transform(X)).head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.500000,-0.866025,0.500051,...,-1.285233,1.493052,-0.295368,-0.547248,0.024951,0.132681,-0.243228,0.284319,-0.136814,0.992642
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.500000,-0.866025,0.500051,...,-0.311136,0.773956,-0.475029,-0.575731,0.025086,0.064942,1.650825,-1.705114,1.230334,0.992642
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.500000,0.866025,0.500051,...,-1.415729,-0.452975,-0.475029,-0.645487,0.023709,0.566901,1.826242,0.284319,0.388141,0.992642
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.866025,0.500000,0.500051,...,-1.121214,0.111738,-0.472667,-0.407736,0.022794,3.102749,1.826242,0.284319,-0.503149,0.992642
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.866025,0.500000,0.500051,...,-0.999177,0.591136,-0.475025,0.876930,0.023413,0.610323,0.386884,0.284319,0.369258,0.992642


model selection

In [9]:
model = None